In [5]:
from simulation_loop import SimulationParams, simulate_many

from scipy.stats import norm, expon, uniform, gamma
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
np.random.seed(42)

In [11]:
class ArrivalDistribution:
    first_round_time = 9
    second_round_time = 15
    def __init__(self, p=0.5, offset=0):
        self.p = p
        self.first_peak = norm(loc=10 + offset, scale=1)
        self.second_peak = norm(loc=15 + offset, scale=1.5)

    def pdf(self, x):
        return self.p * self.first_peak.pdf(x) + (1 - self.p) * self.second_peak.pdf(x)

    def rvs(self, size=()):
        s1 = self.first_peak.rvs(size=size)
        s2 = self.second_peak.rvs(size=size)
        choice = uniform.rvs(size=size) < self.p
        return np.where(choice, s1, s2)
    
discharge_dist = ArrivalDistribution(p=0.55, offset=0)
arrival_dist = ArrivalDistribution(p=0.55, offset=1)

def service_time_dist():
    return 5/60 + expon(scale=1/60).rvs()

def transport_time_dist(distance_meters):
    if distance_meters == 0:
        return 0
    speed = 1.4
    return (distance_meters / speed + expon(scale=1).rvs()) / 3600

distances = np.array([[10, 175], [175, 10]])
arrival_weights = np.array([1.0])
discharge_weights = np.array([1.0])

# Sentivity Analysis : Number of bots

In [12]:
params = SimulationParams(
    discharge_dist=discharge_dist,
    service_time_dist=service_time_dist,
    transport_time_dist=transport_time_dist,
    distance_matrix=distances,
    arrival_weights=arrival_weights,
    discharge_weights=discharge_weights,
    n_elevators=1,
    n_robots=10
)

# Run simulation
df = simulate_many(params, n_iters=10)
print(df.columns)  
print(df.index)


TypeError: SimulationParams.__init__() missing 1 required positional argument: 'arrival_dist'

In [111]:
params = SimulationParams(
    discharge_dist=discharge_dist,
    arrival_dist=arrival_dist,
    service_time_dist=service_time_dist,
    transport_time_dist=transport_time_dist,
    distance_matrix=distances,
    arrival_weights=arrival_weights,
    discharge_weights=discharge_weights,
    n_patients=200,
    n_elevators=1,
    n_robots=25
)

# Run raw simulation
events, data = simulate_system(params)

# Time deltas between events (in hours → minutes)
times = data["times"]
dt_minutes = np.diff(times, prepend=times[0]) * 60  # in minutes

# Threshold and percentiles
threshold_minutes = 30
percentiles = [50, 90, 95, 99]

print(f"Queue time threshold: ≤ {threshold_minutes} minutes at each percentile\n")

# Buffers: dirty and clean (elevator 1)
buffer_metrics = [
    (data["buffers"][Buffer.DIRTY, 1], "Dirty Buffer"),
    (data["buffers"][Buffer.CLEAN, 1], "Clean Buffer"),
    (data["demands"][1], "Bed Demand"),
]

for queue_lengths, label in buffer_metrics:
    time_weighted = np.repeat(queue_lengths, np.round(dt_minutes).astype(int))
    
    print(f"{label}:")
    for p in percentiles:
        q = np.percentile(time_weighted, p)
        wait_minutes = q * 60  # assume 1 bed waited == 1 hour
        status = "✅ OK" if wait_minutes <= threshold_minutes else "❌ Exceeds"
        print(f"  {p:>3}th percentile → {wait_minutes:5.1f} min → {status}")
    print()

Queue time threshold: ≤ 30 minutes at each percentile

Dirty Buffer:
   50th percentile →   0.0 min → ✅ OK
   90th percentile →   0.0 min → ✅ OK
   95th percentile →  60.0 min → ❌ Exceeds
   99th percentile → 120.0 min → ❌ Exceeds

Clean Buffer:
   50th percentile →   0.0 min → ✅ OK
   90th percentile →   0.0 min → ✅ OK
   95th percentile → 180.0 min → ❌ Exceeds
   99th percentile → 360.0 min → ❌ Exceeds

Bed Demand:
   50th percentile → 1620.0 min → ❌ Exceeds
   90th percentile → 5040.0 min → ❌ Exceeds
   95th percentile → 5400.0 min → ❌ Exceeds
   99th percentile → 5520.0 min → ❌ Exceeds

